In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import calendar

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.8f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
from xgboost.sklearn import XGBRegressor
from sklearn.datasets import make_classification
import xgboost as xgb
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from collections import OrderedDict
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

/home/alfonso/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/alfonso/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
train_x = pd.read_csv('train_paso_5.csv')
test_x = pd.read_csv('test_paso_5.csv')
test = pd.read_csv('trocafone_kaggle_test.csv') 

In [4]:
train_x.head()

,person,label,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,mean_time_between_ad,mean_time_between_bl,mean_time_between_st,mean_time_between_se,...,time_to_last_co,time_to_last_ch,time_to_last_le,time_to_last_ad,time_to_last_bl,time_to_last_gl,time_to_last_sp,time_to_last_vs,time_to_last_se,time_to_last_st
0,0566e9c1,0,4.06189002,0.00000000,0.00000000,4.68384428,4.19700473,4.64753456,0.00000000,0.00000000,...,"710,133.00000000","710,451.00000000",-1.00000000,-1.00000000,"710,561.00000000",-1.00000000,-1.00000000,"26,916.00000000",-1.00000000,"631,051.00000000"
1,6ec7ee77,0,0.00000000,-1.00000000,-1.00000000,0.00000000,0.00000000,-1.00000000,-1.00000000,-1.00000000,...,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,"336,875.00000000",-1.00000000,-1.00000000
2,abe7a2fb,0,4.99240895,-1.00000000,0.00000000,5.64788828,5.93325464,5.85617506,-1.00000000,6.20750888,...,-1.00000000,"174,045.00000000",-1.00000000,-1.00000000,"174,349.00000000",-1.00000000,"174,391.00000000","174,582.00000000",-1.00000000,-1.00000000
3,34728364,0,4.31442262,-1.00000000,-1.00000000,5.39360211,-1.00000000,2.42283597,-1.00000000,0.00000000,...,-1.00000000,-1.00000000,-1.00000000,-1.00000000,"525,929.00000000",-1.00000000,-1.00000000,"388,952.00000000",-1.00000000,-1.00000000
4,87ed62de,0,1.94479101,-1.00000000,2.18752072,0.00000000,2.53497746,-1.00000000,-1.00000000,-1.00000000,...,-1.00000000,"1,124,140.00000000",-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,"1,124,607.00000000",-1.00000000,-1.00000000


In [5]:
test_x.head()

,person,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,mean_time_between_ad,mean_time_between_bl,mean_time_between_st,mean_time_between_se,mean_time_between_sp,...,time_to_last_co,time_to_last_ch,time_to_last_le,time_to_last_ad,time_to_last_bl,time_to_last_gl,time_to_last_sp,time_to_last_vs,time_to_last_se,time_to_last_st
0,4886f805,2.23929948,-1.00000000,0.00000000,0.00000000,-1.00000000,-1.00000000,-1.00000000,0.00000000,0.00000000,...,-1.00000000,"1,208,883.00000000",-1.00000000,-1.00000000,-1.00000000,-1.00000000,"1,208,947.00000000","1,209,157.00000000",-1.00000000,-1.00000000
1,0297fc1e,4.32526386,-1.00000000,6.25248470,5.10494935,5.60169127,6.34311660,-1.00000000,-1.00000000,6.17785202,...,-1.00000000,"790,224.00000000","4,198,650.00000000",-1.00000000,"3,986,189.00000000",-1.00000000,"4,480,892.00000000","303,072.00000000",-1.00000000,-1.00000000
2,2d681dd8,4.53048657,-1.00000000,0.00000000,5.92823347,0.00000000,5.32633228,-1.00000000,5.92823347,0.00000000,...,-1.00000000,"361,030.00000000",-1.00000000,-1.00000000,"361,097.00000000",-1.00000000,"361,166.00000000","361,407.00000000",-1.00000000,-1.00000000
3,cccea85e,3.38903466,-1.00000000,0.00000000,4.98811858,5.16420984,5.52115029,4.39299576,4.91259017,0.00000000,...,-1.00000000,"1,730,109.00000000",-1.00000000,-1.00000000,"86,540.00000000",-1.00000000,"40,977.00000000","35,493.00000000",-1.00000000,"1,730,052.00000000"
4,4c8a8b93,3.13094156,-1.00000000,4.70165432,4.26042290,4.29597287,2.74092634,-1.00000000,4.25383454,2.82550726,...,-1.00000000,"1,115,262.00000000",-1.00000000,-1.00000000,"1,126,298.00000000",-1.00000000,"1,202,519.00000000","862,005.00000000",-1.00000000,-1.00000000


In [6]:
#feature engieri del train,misma columna train,pero sin label.

In [7]:
train_y = train_x['label']
train_y.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [8]:
train_x_new = train_x.drop(['label','person'], 1)

In [9]:
train_x_new.head()

,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,mean_time_between_ad,mean_time_between_bl,mean_time_between_st,mean_time_between_se,mean_time_between_sp,mean_time_between_vp,...,time_to_last_co,time_to_last_ch,time_to_last_le,time_to_last_ad,time_to_last_bl,time_to_last_gl,time_to_last_sp,time_to_last_vs,time_to_last_se,time_to_last_st
0,4.06189002,0.00000000,0.00000000,4.68384428,4.19700473,4.64753456,0.00000000,0.00000000,-1.00000000,3.60710881,...,"710,133.00000000","710,451.00000000",-1.00000000,-1.00000000,"710,561.00000000",-1.00000000,-1.00000000,"26,916.00000000",-1.00000000,"631,051.00000000"
1,0.00000000,-1.00000000,-1.00000000,0.00000000,0.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,...,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,"336,875.00000000",-1.00000000,-1.00000000
2,4.99240895,-1.00000000,0.00000000,5.64788828,5.93325464,5.85617506,-1.00000000,6.20750888,6.04419878,5.49300949,...,-1.00000000,"174,045.00000000",-1.00000000,-1.00000000,"174,349.00000000",-1.00000000,"174,391.00000000","174,582.00000000",-1.00000000,-1.00000000
3,4.31442262,-1.00000000,-1.00000000,5.39360211,-1.00000000,2.42283597,-1.00000000,0.00000000,-1.00000000,3.77832745,...,-1.00000000,-1.00000000,-1.00000000,-1.00000000,"525,929.00000000",-1.00000000,-1.00000000,"388,952.00000000",-1.00000000,-1.00000000
4,1.94479101,-1.00000000,2.18752072,0.00000000,2.53497746,-1.00000000,-1.00000000,-1.00000000,-1.00000000,2.24582101,...,-1.00000000,"1,124,140.00000000",-1.00000000,-1.00000000,-1.00000000,-1.00000000,-1.00000000,"1,124,607.00000000",-1.00000000,-1.00000000


In [39]:
modelo_xgb = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.65, learning_rate = 0.1,
                max_depth = 2, gamma = 1.0 , n_estimators = 5000,scale_pos_weight=1, subsample=0.95, min_child_weight=5,
                reg_lambda=0.075,reg_alpha =0.03)

In [40]:
modelo_xgb.fit(train_x_new,train_y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.65,
       gamma=1.0, learning_rate=0.1, max_delta_step=0, max_depth=2,
       min_child_weight=5, missing=None, n_estimators=5000, nthread=-1,
       objective='reg:linear', reg_alpha=0.03, reg_lambda=0.075,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.95)

In [41]:
OrderedDict(sorted(modelo_xgb.booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

OrderedDict([('time_to_last_ch', 58),
             ('time_to_last_vs', 20),
             ('time_to_last_co', 18),
             ('mean_time_between_co', 14),
             ('mean_time_between_se', 14),
             ('febrero', 11),
             ('8GB', 10),
             ('Samsung', 9),
             ('suma_checkout_y_conversion', 8),
             ('time_to_last_st', 8),
             ('mayo', 7),
             ('viewed product', 6),
             ('mean_time_between_ch', 6),
             ('brand listing', 6),
             ('Excelente', 6),
             ('mean_time_between_bl', 5),
             ('Motorola', 5),
             ('multiplicacion_checkout_y_conversion', 5),
             ('checkout', 4),
             ('search engine hit', 4),
             ('time_to_last_vp', 4),
             ('Bom - Sem Touch ID', 4),
             ('visited site', 4),
             ('time_to_last_sp', 4),
             ('mean_time_between_sp', 3),
             ('desconocido_modelo', 3),
             ('less_frequent_ev

In [43]:
#train_x_new = train_x_new.drop(['conversion','less_frequent_event_3','sabado','mean_time_between_ad',\
#                 'most_frequent_event_4','mean_time_between_vp','count_less_frequent_event',\
#                 'mean_per_event','iPhone','jueves','32GB','mean_time_between_vs','abril','16GB','marzo'],1)

In [44]:
columnas=train_x_new.columns

In [45]:
parameters_for_testing = {
#    'colsample_bytree':[0.65],
#    'gamma':[0.9,1,1.1],
#    'min_child_weight':[4,5,6],               # 4
    'learning_rate':[0.03,0.01,0.06,0.1],
#    'max_depth':[2],                                 # 2
#    'n_estimators':[1000,500,750,200],#10,10100,1000
#   'reg_alpha':[0.03],#1e-5, 1e-2
#      'reg_lambda':[0.1,0.01,0.05,0.07,0.03],
#    'subsample':[0.95]#0.6
} 

In [46]:
gsearch1 = GridSearchCV(estimator = modelo_xgb, param_grid = parameters_for_testing,\
                        n_jobs=4,iid=False, verbose=10,scoring='roc_auc',refit=True)
gsearch1.fit(train_x_new,train_y)
print (gsearch1.grid_scores_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] learning_rate=0.03 ..............................................
[CV] learning_rate=0.03 ..............................................
[CV] learning_rate=0.03 ..............................................
[CV] learning_rate=0.01 ..............................................
[CV] ..................... learning_rate=0.03, score=0.861995 - 8.4min
[CV] learning_rate=0.01 ..............................................
[CV] ..................... learning_rate=0.01, score=0.862853 - 8.4min
[CV] learning_rate=0.01 ..............................................
[CV] ..................... learning_rate=0.03, score=0.865782 - 8.4min
[CV] learning_rate=0.06 ..............................................
[CV] ..................... learning_rate=0.03, score=0.849488 - 8.5min
[CV] learning_rate=0.06 ..............................................
[CV] ..................... learning_rate=0.01, score=0.865591 - 8.6min
[CV] learning_rat

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed: 17.0min


[CV] ..................... learning_rate=0.01, score=0.849104 - 8.7min
[CV] learning_rate=0.1 ...............................................
[CV] ..................... learning_rate=0.06, score=0.849799 - 8.6min
[CV] learning_rate=0.1 ...............................................


[Parallel(n_jobs=4)]: Done   7 out of  12 | elapsed: 17.1min remaining: 12.2min


[CV] ..................... learning_rate=0.06, score=0.860528 - 8.7min
[CV] learning_rate=0.1 ...............................................
[CV] ...................... learning_rate=0.1, score=0.858989 - 8.9min


[Parallel(n_jobs=4)]: Done   9 out of  12 | elapsed: 25.9min remaining:  8.6min


[CV] ..................... learning_rate=0.06, score=0.865331 - 8.9min
[CV] ...................... learning_rate=0.1, score=0.865984 - 8.9min
[CV] ...................... learning_rate=0.1, score=0.848151 - 8.9min


[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed: 26.0min finished


[mean: 0.85909, std: 0.00696, params: {'learning_rate': 0.03}, mean: 0.85918, std: 0.00721, params: {'learning_rate': 0.01}, mean: 0.85855, std: 0.00649, params: {'learning_rate': 0.06}, mean: 0.85771, std: 0.00734, params: {'learning_rate': 0.1}]
best params
{'learning_rate': 0.01}
best score
0.8591825830791181


In [39]:
#mejor resultado paso 5:
#objective ='reg:linear', \
#                colsample_bytree = 0.8, learning_rate = 0.03,\
#                max_depth = 2, gamma = 0.0 , n_estimators = 750, min_child_weight=4,\
#                reg_alpha=0.001, reg_lambda=0.005 , subsample= 0.8,scale_pos_weight=1

In [35]:
#creo el mejor modelo:
mejor_modelo_xgb = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.6, learning_rate = 0.1,
                max_depth = 1, gamma = 0.5 , n_estimators = 500,scale_pos_weight=1, subsample=0.9, min_child_weight=3,
                  reg_lambda=0.1, reg_alpha=0.08)

In [36]:
#entreno el mejor modelo
mejor_modelo_xgb.fit(train_x_new,train_y)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0.0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=6, missing=None, n_estimators=700, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8)

In [37]:
# ahora manejo el set para predecir

In [38]:
test_x = test_x.loc[:,columnas]


In [39]:
test_x.head()

,region_mean,city_mean,country_mean,device_mean,engine_mean,system_mean,mean_time_between_events,mean_time_between_co,mean_time_between_ch,mean_time_between_vs,...,time_to_first_ch,time_to_first_le,time_to_first_ad,time_to_first_bl,time_to_first_gl,time_to_first_sp,time_to_first_vs,time_to_first_se,time_to_first_st,browser_mean
0,0.08102395,0.00000000,0.08829165,0.07383724,0.07536082,0.05016195,2.23929948,-1.00000000,0.00000000,0.00000000,...,"1,208,883.00000000",-1.00000000,-1.00000000,-1.00000000,-1.00000000,"1,208,947.00000000","1,209,157.00000000",-1.00000000,-1.00000000,0.10935961
1,0.08102395,0.09150327,0.08829165,0.07383724,-1.00000000,0.06928492,4.32526386,-1.00000000,6.25248470,5.10494935,...,"11,521,119.00000000","4,198,650.00000000",-1.00000000,"10,596,743.00000000",-1.00000000,"12,011,361.00000000","12,272,605.00000000",-1.00000000,-1.00000000,0.10935961
2,0.08189670,0.04128440,0.08829165,0.10271789,0.07536082,0.09108108,4.53048657,-1.00000000,0.00000000,5.92823347,...,"361,030.00000000",-1.00000000,-1.00000000,"1,209,090.00000000",-1.00000000,"361,166.00000000","1,209,090.00000000",-1.00000000,-1.00000000,0.07856738
3,0.08189670,0.04683196,0.08829165,0.10271789,0.07536082,0.10184407,3.38903466,-1.00000000,0.00000000,4.98811858,...,"1,730,109.00000000",-1.00000000,-1.00000000,"2,078,596.00000000",-1.00000000,"40,977.00000000","2,078,820.00000000",-1.00000000,"1,828,920.00000000",0.07856738
4,0.08189670,0.01779359,0.08829165,0.07383724,0.07536082,0.05016195,3.13094156,-1.00000000,4.70165432,4.26042290,...,"1,165,572.00000000",-1.00000000,-1.00000000,"1,130,153.00000000",-1.00000000,"1,207,872.00000000","1,208,085.00000000",-1.00000000,-1.00000000,0.05641531


In [28]:
#predecimos
prediction = mejor_modelo_xgb.predict(test_x)

In [29]:
ids = test['person'].values
dataKaggle = pd.DataFrame(ids ,columns={'person'})
dataKaggle.head(5)

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [30]:
#Aca agregamos una columna que seria se postulop
dataKaggle['label'] = prediction

In [31]:
dataKaggle.nlargest(5,'label')

,person,label
4520,b580ba86,0.99691373
485,531a6385,0.92387319
12973,52e7e9bd,0.92335427
19379,b2fafa1e,0.90164638
3263,83c12101,0.86568952


In [32]:
dataKaggle.head()

,person,label
0,4886f805,-0.03713733
1,0297fc1e,-0.07494235
2,2d681dd8,0.02683535
3,cccea85e,0.15057865
4,4c8a8b93,-0.00635821


In [33]:
dataKaggle['label'] = dataKaggle['label'].transform(lambda x: 0.0 if x < 0.0 else x)
dataKaggle['label'] = dataKaggle['label'].transform(lambda x: 1.0 if x > 1.0 else x)

In [34]:
dataKaggle.to_csv("predictionXGBoost.csv", index = False)